In [2]:
from transformers import DalleBartForConditionalGeneration, DalleBartProcessor
import torch
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
import torchvision.transforms as transforms

path = './pics'

# Custom dataset class
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.images = os.listdir(img_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

# Load pre-trained model
model = DalleBartForConditionalGeneration.from_pretrained(path)
processor = DalleBartProcessor.from_pretrained(path)

# Dataset and DataLoader
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])
dataset = CustomImageDataset(img_dir=path, transform=transform)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

model.train()
for epoch in range(1):
    for images in dataloader:
        images = images.to(device)
        outputs = model(images)  # You need to modify how inputs/outputs are handled based on the model specifics
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

/workspaces/ML-inter_task/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'DalleBartForConditionalGeneration' from 'transformers' (/workspaces/ML-inter_task/.venv/lib/python3.10/site-packages/transformers/__init__.py)